In [7]:
import numpy as np
import tensorflow as tf
import time
from tqdm import tqdm

# add package to the path
import os
import sys

SEED = 42

tffm_path = '../'
print('Add path : ', os.listdir(tffm_path))
sys.path.append(tffm_path)
print()
print('path pacakge path : ', sys.path)

print('tensorflow version : ', tf.__version__)

Add path :  ['.DS_Store', 'LICENSE', 'gpu_benchmark.ipynb', 'test.py', 'README.md', 'example.ipynb', 'yltsai', 'setup.py', '.gitignore', 'tffm', 'setup.cfg', '.ipynb_checkpoints', '.git']

path pacakge path :  ['/Users/YuLong/Desktop/Working_Area/recsys_im/tffm/yltsai', '/usr/local/spark/python', '/Users/YuLong/Desktop/Working_Area/recsys_im/tffm/yltsai', '/Users/YuLong/miniconda3/envs/py_37_ds/lib/python37.zip', '/Users/YuLong/miniconda3/envs/py_37_ds/lib/python3.7', '/Users/YuLong/miniconda3/envs/py_37_ds/lib/python3.7/lib-dynload', '', '/Users/YuLong/miniconda3/envs/py_37_ds/lib/python3.7/site-packages', '/Users/YuLong/Desktop/Working_Area/DataScience_Note/demo/packaging', '/Users/YuLong/Desktop/Working_Area/pose_recognition', '/Users/YuLong/miniconda3/envs/py_37_ds/lib/python3.7/site-packages/IPython/extensions', '/Users/YuLong/.ipython', '../', '../', '../']
tensorflow version :  1.14.0


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
# from sklearn.datasets import fetch_mldata
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

print('-----END IMPORT MESSAGE------ \n\n\n')

# dataset = fetch_openml("mnist_784")
mnist = input_data.read_data_sets("MNIST_data/")

mnist_images = mnist.train.images
mnist_labels = mnist.train.labels

n_three, n_five = sum(mnist_labels==3), sum(mnist_labels==5)

X_all = np.vstack([
    mnist_images[mnist_labels==3,:],
    mnist_images[mnist_labels==5,:]
])

y_all = np.array([1] * n_three + [0] * n_five)
# make it more sparse
X_all = X_all * (np.random.uniform(0, 1, X_all.shape) > 0.8)


print('Dataset shape: {}'.format(X_all.shape))
print('Non-zeros rate: {:.05f}'.format(np.mean(X_all != 0)))
print('Classes balance: {:.03f} / {:.03f}'.format(np.mean(y_all==0), np.mean(y_all==1)))

X_tr, X_te, y_tr, y_te = train_test_split(X_all, y_all, random_state=42, test_size=0.3)

-----END IMPORT MESSAGE------ 



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Dataset shape: (10625, 784)
Non-zeros rate: 0.04034
Classes balance: 0.469 / 0.531


# dense

In [8]:
from tffm import TFFMClassifier

for order in [2, 3]:
    model = TFFMClassifier(
        order=order, 
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='dense',
        seed=42
    )
    model.fit(X_tr, y_tr, show_progress=True)
    predictions = model.predict(X_te)
    print('[order={}] accuracy: {}'.format(order, accuracy_score(y_te, predictions)))
    # this will close tf.Session and free resources
    model.destroy()  

100%|██████████| 50/50 [00:02<00:00, 22.99epoch/s]
/Users/YuLong/miniconda3/envs/py_37_ds/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


[order=2] accuracy: 0.8760978670012547


100%|██████████| 50/50 [00:04<00:00, 10.02epoch/s]


[order=3] accuracy: 0.9099749058971142


# sparse

In [9]:
import scipy.sparse as sp
# only CSR format supported
X_tr_sparse = sp.csr_matrix(X_tr)
X_te_sparse = sp.csr_matrix(X_te)

In [10]:
for order in [2, 3]:
    model = TFFMClassifier(
        order=order, 
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        input_type='sparse',
        seed=42
    )
    model.fit(X_tr_sparse, y_tr, show_progress=True)
    predictions = model.predict(X_te_sparse)
    print('[order={}] accuracy: {}'.format(order, accuracy_score(y_te, predictions)))
    model.destroy()

100%|██████████| 50/50 [00:09<00:00,  5.12epoch/s]
/Users/YuLong/miniconda3/envs/py_37_ds/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


[order=2] accuracy: 0.8760978670012547


100%|██████████| 50/50 [00:17<00:00,  2.90epoch/s]


[order=3] accuracy: 0.9099749058971142


# Why Sparse matrix is slower?

* setting : 

```
    model = TFFMClassifier(
        order=order, 
        rank=10, 
        optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
        n_epochs=50, 
        batch_size=1024,
        init_std=0.001,
        reg=0.01,
        seed=42
    )
```

|type|speed(epoch/s)|
|----|--------------|
|dense - order 2|22.9|
|sparse - order 2|5.12|
|dense - order 3|10.0|
|sparse - order 3|2.9|

* related operation

1. `matmul_wrapper`

```python
def matmul_wrapper(A, B, optype):
    """Wrapper for handling sparse and dense versions of `tf.matmul` operation.

    Parameters
    ----------
    A : tf.Tensor
    B : tf.Tensor
    optype : str, {'dense', 'sparse'}

    Returns
    -------
    tf.Tensor
    """
    with tf.name_scope('matmul_wrapper') as scope:
        if optype == 'dense':
            return tf.matmul(A, B)
        elif optype == 'sparse':
            return tf.sparse_tensor_dense_matmul(A, B)
        else:
            raise NameError('Unknown input type in matmul_wrapper')

```
2. `pow_wrapper`

```python
def pow_wrapper(X, p, optype):
    """Wrapper for handling sparse and dense versions of `tf.pow` operation.

    Parameters
    ----------
    X : tf.Tensor
    p : int
    optype : str, {'dense', 'sparse'}

    Returns
    -------
    tf.Tensor
    """
    with tf.name_scope('pow_wrapper') as scope:
        if optype == 'dense':
            return tf.pow(X, p)
        elif optype == 'sparse':
            return tf.SparseTensor(X.indices, tf.pow(X.values, p), X.dense_shape)
        else:
            raise NameError('Unknown input type in pow_wrapper')

```
3. `count_non_zero_wrapper`

```python
def count_nonzero_wrapper(X, optype):
    """Wrapper for handling sparse and dense versions of `tf.count_nonzero`.

    Parameters
    ----------
    X : tf.Tensor (N, K)
    optype : str, {'dense', 'sparse'}

    Returns
    -------
    tf.Tensor (1,K)
    """
    with tf.name_scope('count_nonzero_wrapper') as scope:
        if optype == 'dense':
            return tf.count_nonzero(X, axis=0, keep_dims=True)
        elif optype == 'sparse':
            indicator_X = tf.SparseTensor(X.indices, tf.ones_like(X.values), X.dense_shape)
            return tf.sparse_reduce_sum(indicator_X, axis=0, keep_dims=True)
        else:
            raise NameError('Unknown input type in count_nonzero_wrapper')
```

初步診斷可能的Bottleneck

1. `sparse_tensor_dense_matmul`
2. `tf.SparseTensor(X.indices, tf.pow(X.values, p), X.dense_shape)`
3. `tf.SparseTensor`
4. `sparse_reduce_sum`


reference

keyword : tensorflow sparse matrix slow 
[how-to-perform-efficient-sparse-matrix-multiplication-by-using-tf-matmul](https://stackoverflow.com/questions/44859321/how-to-perform-efficient-sparse-matrix-multiplication-by-using-tf-matmul)

[Tensorflow两个稀疏矩阵相乘](https://zhuanlan.zhihu.com/p/150920129)

[tensorflow-why-passing-dense-matrix-is-noticeably-faster-than-sparse](https://stackoverflow.com/questions/47392441/tensorflow-why-passing-dense-matrix-is-noticeably-faster-than-sparse)